In [29]:
# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np
import tqdm as tqdm
from time import sleep
import random

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

In [42]:
ciudades_espana = [
    "Madrid",
    "Barcelona",
    "Valencia",
    "Sevilla",
    "Zaragoza",
    "Málaga",
    "Murcia",
    "Palma de Mallorca",
    "Las Palmas de Gran Canaria",
    "Bilbao",
    "Alicante",
    "Córdoba",
    "Valladolid",
    "Vigo",
    "Gijón",
    "L'Hospitalet de Llobregat",
    "La Coruña",
    "Granada",
    "Elche",
]



In [33]:
def obtenertabla(sopatabla,ciudad,mes):
    lista_df=[]
    for subtabla in sopatabla:
        df=pd.DataFrame([subtabla.findChildren('tr',recursive=False)[i].text for i in range(len(subtabla.findChildren('tr',recursive=False)))])
        lista_df.append(df)
    df_final=pd.concat(lista_df, axis=1)
    df_final.columns=['Dia','Temperature','Dew Point','Humidity','Wind Speed','Pressure','Precipitation']
    df_final1=df_final.iloc[1:]
    reemplazos={'-1': 'Enero', '-2': 'Febrero', '-3': 'Marzo', '-4': 'Abril', '-5': 'Mayo', '-6': 'Junio', '-7': 'Julio', '-8': 'Agosto', '-9': 'Septiembre', '10': 'Octubre', '11': 'Noviembre', '12': 'Diciembre'}
    df_final1['Mes']=mes
    df_final1['Mes'] = df_final1['Mes'].replace(reemplazos)
    df_final1['Ubicación']=ciudad
    df_final1[['Temp Max', 'Temp Avg', 'Temp Min']] = df_final1['Temperature'].str.split(expand=True)
    df_final1.drop(columns='Temperature',inplace=True)
    df_final1[['DP Max', 'DP Avg', 'DP Min']] = df_final1['Dew Point'].str.split(expand=True)
    df_final1.drop(columns='Dew Point',inplace=True)
    df_final1[['Hum Max', 'Hum Avg', 'Hum Min']] = df_final1['Humidity'].str.split(expand=True)
    df_final1.drop(columns='Humidity',inplace=True)
    df_final1[['WS Max', 'WS Avg', 'WS Min']] = df_final1['Wind Speed'].str.split(expand=True)
    df_final1.drop(columns='Wind Speed',inplace=True)
    df_final1[['Pres Max', 'Pres Avg', 'Pres Min']] = df_final1['Pressure'].str.split(expand=True)
    df_final1.drop(columns='Pressure',inplace=True)
    columna_a_mover = df_final1['Precipitation']
    df_final1.drop(columns='Precipitation',inplace=True) 
    df_final1['Precipitation']= columna_a_mover 
    return df_final1

In [43]:
driver=webdriver.Chrome()
url_wunder = "https://www.wunderground.com/history"
    # Abrimos navegador y maximizamos página en la url
driver.get(url_wunder)
driver.maximize_window()
    # Aceptacion de cookies
sleep(2)
iframe=WebDriverWait(driver,10).until(EC.presence_of_element_located(('xpath','//*[@id="sp_message_iframe_1165301"]')))
driver.switch_to.frame(iframe)
    #seleccionar busqueda
try:
    driver.find_element("css selector","#notice > div.message-component.message-row.cta-buttons-container > div.message-component.message-column.cta-button-column.reject-column > button").click()
except:    
    print('No encuentro el boton')
sleep(2)
driver.switch_to.default_content()
listatablasciudades=[]
ciudadesrandom=random.sample(ciudades_espana,5)
print(ciudadesrandom)
for ciudad in ciudadesrandom:
    try:
        driver.find_element("css selector","#historySearch").send_keys({ciudad}, Keys.TAB)
    except:    
        print('No encuentro el boton')
    sleep(2)
    # Pulsar boton View
    try:
        driver.find_element("css selector","#dateSubmit").click()
    except:    
        print('No encuentro el boton View')
    sleep(2)
    # Pulsar boton View
    try:
        driver.find_element("css selector","#dateSubmit").click()
    except:    
        print('No encuentro el boton View')
    sleep(4)
    # Pulsar boton Monthly
    try:
        driver.find_element("css selector","#inner-content > div.region-content-main > div.row > div:nth-child(1) > div:nth-child(1) > div > lib-link-selector > div > div > div > a:nth-child(3)").click()
    except:    
        print('No encuentro el boton de Monthly')
    sleep(2)
    urlactual= driver.current_url    # Obtener url actual
    listaurls=[]
    for dia in range(1, 11): # Obtener bucle de urls modificado
        url_modificada = urlactual[:-2] + str(dia)
        listaurls.append(url_modificada)
    listatablas=[]
    for url in listaurls: #obtener de la lista de urls las tablas de cada url
        try:
            driver.get(url)
            mes = url[-2:]
            sleep(3)
            html_table_page = driver.page_source
            sopa = BeautifulSoup(html_table_page, 'html.parser')
            tabla = sopa.findAll("table", {'aria-labelledby': 'Days data'})
            if not tabla:  # Verificar si no se encontró ninguna tabla
                print(f'No se encontró tabla para la URL: {url}')
                continue  # Salta a la siguiente URL

            tabla1 = obtenertabla(tabla, ciudad, mes)
            
            if tabla1 is not None:  # Verificar que se haya obtenido un DataFrame
                listatablas.append(tabla1)
            else:
                print(f'Error al obtener tabla para la ciudad {ciudad} en el mes {mes}')
            
        except:
            print('Error al procesar la URL')

    # Solo intentar concatenar si hay tablas
    if listatablas:  # Verificar que la lista no esté vacía
        df_unido = pd.concat(listatablas, ignore_index=True)
        listatablasciudades.append(df_unido)
        driver.get(url_wunder)
    else: #Si la lista esta vacía se imprime que no se encontraron datos
        print(f'No se encontraron tablas de datos de {ciudad}')
        driver.get(url_wunder)

df_unidofinal=pd.concat(listatablasciudades, ignore_index=True)

['Madrid', 'Murcia', 'Gijón', 'Elche', 'Granada']


C:\Users\alexc\AppData\Local\Temp\ipykernel_18632\3888738003.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final1['Mes']=mes
C:\Users\alexc\AppData\Local\Temp\ipykernel_18632\3888738003.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final1['Mes'] = df_final1['Mes'].replace(reemplazos)
C:\Users\alexc\AppData\Local\Temp\ipykernel_18632\3888738003.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-1
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-2
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-3
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-4
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-5
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-6
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-7
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-8
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-9
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/LEMI/date/2024-10
No se encontraron tablas de datos de Mu

C:\Users\alexc\AppData\Local\Temp\ipykernel_18632\3888738003.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final1['Mes']=mes
C:\Users\alexc\AppData\Local\Temp\ipykernel_18632\3888738003.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final1['Mes'] = df_final1['Mes'].replace(reemplazos)
C:\Users\alexc\AppData\Local\Temp\ipykernel_18632\3888738003.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-1
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-2
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-3
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-4
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-5
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-6
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-7
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-8
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-9
No se encontró tabla para la URL: https://www.wunderground.com/history/monthly/SAVY/date/2024-10
No se encontraron tablas de datos de El

In [44]:
# df_unidofinal

array(['Madrid', 'Gijón'], dtype=object)

In [159]:
# df_final=pd.concat(lista_df, axis=1)
# df_final.columns=['Dia','Temperature','Dew Point','Humidity','Wind Speed','Pressure','Precipitation']
# df_final1=df_final.iloc[1:]

In [167]:
# df_final1[['Temp Max', 'Temp Avg', 'Temp Min']] = df_final1['Temperature'].str.split(expand=True)
# df_final1.drop(columns='Temperature',inplace=True)
# df_final1[['DP Max', 'DP Avg', 'DP Min']] = df_final1['Dew Point'].str.split(expand=True)
# df_final1.drop(columns='Dew Point',inplace=True)
# df_final1[['Hum Max', 'Hum Avg', 'Hum Min']] = df_final1['Humidity'].str.split(expand=True)
# df_final1.drop(columns='Humidity',inplace=True)
# df_final1[['WS Max', 'WS Avg', 'WS Min']] = df_final1['Wind Speed'].str.split(expand=True)
# df_final1.drop(columns='Wind Speed',inplace=True)
# df_final1[['Pres Max', 'Pres Avg', 'Pres Min']] = df_final1['Pressure'].str.split(expand=True)
# df_final1.drop(columns='Pressure',inplace=True)
# columna_a_mover = df_final1['Precipitation']
# df_final1.drop(columns='Precipitation',inplace=True) 
# df_final1['Precipitation']= columna_a_mover 